In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras import layers, models


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message']) # Added sep='\t', header=None, and names
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

x_train = train_df['message']
y_train = train_df['label']

x_test = test_df['message']
y_test = test_df['label']

In [ ]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

label_map = {'ham': 0, 'spam': 1}
y_train = y_train.map(label_map).astype('float32')
y_test = y_test.map(label_map).astype('float32')


y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
vectorizer = keras.layers.TextVectorization(max_tokens=10000, output_sequence_length=100)
vectorizer.adapt(x_train)

In [ ]:
model = keras.Sequential([
    vectorizer,
    layers.Embedding(input_dim=10000, output_dim=64),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

In [ ]:
if y_test.dtype != 'int64':
    y_test = y_test.astype('int64')

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
new_messages = ["Congratulations! You've won a $1000 Walmart gift card. Click here to claim."]
new_messages = tf.constant(new_messages)

predictions = model.predict(new_messages)
print(predictions[0][0])


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = []
  pred_tensor = tf.constant([pred_text])


  result = model.predict(pred_tensor)

  prediction.append(result[0][0])
  if result[0][0] >= 0.5:
    prediction.append('spam')
  else:
    prediction.append('ham')


  return (prediction)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)

    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
